In [ ]:
"""
Code Description:
Calculates the IM residuals from Cybershake simulations and Empirical calculations

Author: Morteza
Version History:
- Version 1.0: December 20, 2024
"""

# Import the dependencies

In [1]:
import os
import sys
from pathlib import Path
import h5py
import shutil
import pickle

import pandas as pd
import numpy as np
import xarray as xr
from tqdm import tqdm

from concurrent.futures import ThreadPoolExecutor

sys.path.append(str(Path.cwd().parent))

from utils import util

# Path and directory works

In [2]:
root_path = os.getcwd()

data_version = "combined_v20p6_v24p8"
sim_dir = f"/mnt/hypo_data/mab419/Cybershake_Data/{data_version}" #dpath to simulation data
emp_dir = f"/mnt/hypo_data/mab419/Empirical_Data/{data_version}"  # dpath to emirical calculation data
res_dir = f"/mnt/hypo_data/mab419/Residual_Data/{data_version}"  # dpath to residual pickle data
plot_dir = "/mnt/hypo_data/mab419/mab419_cybershake_investigation/usages/plots/"

base_dir = "/mnt/hypo_data/mab419/mab419_cybershake_investigation/base_data"  # dpath to dirctory of base data

# File names of the stations data
stations_ll = "non_uniform_whole_nz_with_real_stations-hh400_v20p3_land.ll"
stations_vs30 = "non_uniform_whole_nz_with_real_stations-hh400_v20p3_land.vs30"
stations_z = "non_uniform_whole_nz_with_real_stations-hh400_v20p3_land.z"

# File  name of 'Fault-Site-Source' database
site_source_db = "new_flt_site_source.db"

# Reading Simulation Data

In [4]:
sim_data_dic = util.load_sim_data(
    sim_dir=sim_dir,
    # faults=["Tauranga05"],
    component="geom",
)

100%|██████████| 478/478 [10:21<00:00,  1.30s/it]


In [ ]:
sim_statistic_dic = util.calc_sim_statistics(
    sim_dir=sim_dir,
    # faults=["FiordSZ03"],
    component="geom",
    proc_flag=False,
)

In [ ]:
#sample
tt = sim_statistic_dic["FiordSZ03"]
temp_df = tt.sel(statistics="mean").to_pandas()
temp_df

# Reading Empirical Data

In [ ]:
emp_statistic_dic = util.load_emp_data(
    emp_dir=emp_dir,
    # faults=["FiordSZ03"],
    component="geom",
    proc_flag= False,
)

# Calculate Residuals

## NOT Normalized Residuals

In [ ]:
residual_dic_not_normal = util.calc_mean_residual(
    sim_dic=sim_statistic_dic,
    emp_dic=emp_statistic_dic,
    faults=[],
    IMs=[],
    normalized=False,
)

In [ ]:
# Sample
residual_dic_not_normal["FiordSZ03"].loc[:, "PGV"].plot()

## Normalized Residuals

In [ ]:
residual_dic_normal = util.calc_mean_residual(
    sim_dic=sim_statistic_dic,
    emp_dic=emp_statistic_dic,
    faults=[],
    IMs=[],
    normalized=True,
)

In [ ]:
# sample
residual_dic_normal["FiordSZ03"].loc[:, "PGV"].plot()

# Obtian Rrup-Residual Relation

## Loading station-r-residual dict from previous pickle file

In [4]:
# Import file name
file_name = "Station_Rrup_Res_Dic.pkl"
file_path = os.path.join(res_dir, file_name)
with open(file_path, "rb") as file:
    Station_Rrup_Res_Dic = pickle.load(file)

print("Dictionary loaded successfully")

Dictionary loaded successfully


## Calculating new dictionary

### Claculating Station_Rrup_Residula Dictionary

In [ ]:
# Derermine which residuals to proceed
Resduals_Results_Dic = residual_dic_not_normal
# Resduals_Results_Dic = residual_dic_normal

Faults = list(Resduals_Results_Dic.keys())  
db_file_path = os.path.join(base_dir, site_source_db)

IMs = ["PGV"]  # Replace with your intensity measures
IMs = [
    "pSA_0.01",
    "pSA_0.02",
    "pSA_0.03",
    "pSA_0.04",
    "pSA_0.05",
    "pSA_0.075",
    "pSA_0.1",
    "pSA_0.12",
    "pSA_0.15",
    "pSA_0.17",
    "pSA_0.2",
    "pSA_0.25",
    "pSA_0.3",
    "pSA_0.4",
    "pSA_0.5",
    "pSA_0.6",
    "pSA_0.7",
    "pSA_0.75",
    "pSA_0.8",
    "pSA_0.9",
    "pSA_1.0",
    "pSA_1.25",
    "pSA_1.5",
    "pSA_2.0",
    "pSA_2.5",
    "pSA_3.0",
    "pSA_4.0",
    "pSA_5.0",
    "pSA_6.0",
    "pSA_7.5",
    "pSA_10.0",
]

Station_Rrup_Res_Dic = {}

# Precompute fault mapping
fault_mapping = util.fault_mapping_from_flt_site_source_db(db_file_path)

# Open the HDF5 file once
with h5py.File(db_file_path, "r") as file_handle:
    for im in IMs:
        print(f"IM= {im}")
        if im not in Station_Rrup_Res_Dic:
            Station_Rrup_Res_Dic[im] = {}

        # Process each fault in parallel
        with ThreadPoolExecutor() as executor:
            results = executor.map(
                lambda fault: (
                    fault,
                    Resduals_Results_Dic[fault][im],  # Residual data
                    util.parallel_multi_station_get_distance_from_db(
                        stations=list(
                            Resduals_Results_Dic[fault][im].index
                        ),  # Extract stations
                        fault_name=fault,
                        file_handle=file_handle,
                        fault_mapping=fault_mapping,
                    ),
                ),
                Faults,
            )

            # Store results in the dictionary
            for fault, res_df, station_rrups_df in results:
                # Merge residuals and distances
                station_rrup_res_df = pd.merge(
                    station_rrups_df, res_df.reset_index(), on="station", how="left"
                )
                station_rrup_res_df = station_rrup_res_df.dropna(subset=["rrup"])
                Station_Rrup_Res_Dic[im][fault] = station_rrup_res_df

In [ ]:
# Sample
Station_Rrup_Res_Dic['pSA_0.01']['HikHBaymax']

### Saving the calculated station-r-residual dict to pickle file

In [ ]:
# Check if the directory exists
if Path(res_dir).exists():
    prompt = input(
        f"The following path already exists! Do you want to delete and renew (1) or terminate (2)? \n{res_dir}\n"
    )
    if prompt == "1":
        # Remove the folder
        shutil.rmtree(res_dir)
        print(f"Deleted and renewed the path: \n{res_dir}")
        os.makedirs(res_dir, exist_ok=False)
    elif prompt == "2":
        print("Terminating the process.")
        exit()
    else:
        print("Invalid input. Terminating the process.")
        exit()
else:
    os.makedirs(res_dir, exist_ok=False)
    print(f"Created the path: \n{res_dir}")

# File path to save the dictionary
file_name = "Station_Rrup_Res_Dic.pkl"
file_path = os.path.join(res_dir, file_name)

# Save the dictionary
with open(file_path, "wb") as file:
    pickle.dump(Station_Rrup_Res_Dic, file)

print(f"Dictionary saved to {file_path}")